In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_credits = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_credits.csv')
df_movies = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv')

In [ ]:
df_credits.head()

In [ ]:
df_movies.head()

In [ ]:
df_credits.rename(columns = {'movie_id':'id'}, inplace = True)

In [ ]:
df_merged = pd.merge(df_credits, df_movies, on = ['id','title'])

In [ ]:
df_merged.head(1)

### Casting the Json Columns

In [ ]:
from ast import literal_eval

json_cols = ['cast', 'crew', 'genres', 'keywords','production_companies', 'production_countries','spoken_languages']

for col in json_cols:
    df_merged[col] = df_merged[col].apply(literal_eval)

In [ ]:
df_merged.cast[0]

### Extrating the features from Json Columns

1. Genres list (from Genres column)
2. Jobs (from Crew column)
3. Percentage of voice artists among total cast (from cast column)

In [ ]:
def get_genre(x):
    if(isinstance(x, list)):
        genre = [i['name'] for i in x]
    
    return genre

def get_jobs(x):
    if(isinstance(x, list)):
        jobs = [i['job'] for i in x]
    return jobs

def get_characternames(x):
    if(isinstance(x, list)):
        chr_name = [i['character'] for i in x]
        countc = 0
        for j in chr_name:
            if('(voice)' in j):
                countc += 1
        if(len(chr_name)!=0):
            return (countc/len(chr_name))
        else:
            return 0

In [ ]:
df_merged['genres'] = df_merged['genres'].apply(get_genre)
df_merged['crew_jobs'] = df_merged['crew'].apply(get_jobs)
df_merged['percent_of_voice_artists'] = df_merged['cast'].apply(get_characternames)

In [ ]:
df_merged.head(1)

In [ ]:
df_merged.dtypes

In [ ]:
np.unique(df_merged.percent_of_voice_artists)

Rounding off the percentage to 3 decimal places

In [ ]:
for x in range(0,len(df_merged['percent_of_voice_artists'])):
    df_merged['percent_of_voice_artists'][x] = np.round(df_merged['percent_of_voice_artists'][x],3)

In [ ]:
np.unique(df_merged['percent_of_voice_artists'])